# MODNet (v0.1.10)

For now, this benchmark file simply loads our existing full benchmark results (265 MB) from [ml-evs/modnet-matbench](https://github.com/ml-evs/modnet-matbench) and exports them in the matbench format. Code for featurisation, hyperparameter optimisation and the final predictions themselves can be found in the aforementioned repository.

In [13]:
def download_and_extract(url, fname):
    import urllib
    from zipfile import ZipFile
    import os
    if os.path.exists(fname):
        print(f"File {fname} already found, will not redownload.")
        return
    
    response = urllib.request.urlretrieve(url, fname)
    with ZipFile(fname, "r") as _zip:
        _zip.extractall(".")
        
version = "0.1"
fname = f"modnet-matbench-{version}"
url = f"https://github.com/ml-evs/modnet-matbench/archive/refs/tags/v{version}.zip"

download_and_extract(url, fname + ".zip")

File modnet-matbench-0.1.zip already found, will not redownload.


In [14]:
import pickle
import numpy as np
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

mb = MatbenchBenchmark(
    autoload=False, 
    subset=[
        'matbench_dielectric', 
        'matbench_jdft2d', 
        'matbench_steels', 
        'matbench_expt_gap', 
        'matbench_phonons',
        'matbench_log_gvrh',
        'matbench_log_kvrh',
        'matbench_glass', 
        # 'matbench_expt_is_metal', 
    ],
)

results_locs = {task.dataset_name: f"{fname}/{task.dataset_name}/results/{task.dataset_name}_results.pkl" for task in mb.tasks}
results_locs["matbench_log_gvrh"] = results_locs["matbench_log_kvrh"] = f"{fname}/matbench_elastic/results/matbench_elastic_results.pkl"
target_key_map = {"matbench_log_gvrh": "log10G_VRH", "matbench_log_kvrh": "log10K_VRH"}

2021-08-02 17:40:59 INFO     Initialized benchmark 'matbench_v0.1' with 8 tasks: 
['matbench_dielectric',
 'matbench_jdft2d',
 'matbench_steels',
 'matbench_expt_gap',
 'matbench_phonons',
 'matbench_log_gvrh',
 'matbench_log_kvrh',
 'matbench_glass']


In [15]:
for task in mb.tasks:

    task.load()
    
    with open(results_locs[task.dataset_name], "rb") as f:
        results = pickle.load(f)
        
    for fold_ind, fold in enumerate(task.folds):

        # Classification tasks require different loading as two values are reported per prediction
        if task.metadata.task_type == CLF_KEY:
            predictions = results["predictions"][fold_ind].values[:, 0].astype(bool).flatten()
        else:
            # Handle predictions that were made with joint/multitarget learning
            if task.dataset_name in target_key_map:
                predictions = results["predictions"][fold_ind][target_key_map[task.dataset_name]].values.flatten()
            else:
                predictions = results["predictions"][fold_ind].values.flatten()

            
        task.record(fold, predictions)
        
    if task.metadata.task_type == CLF_KEY:
        print(f"{task.dataset_name}: Accuracy score {task.scores['accuracy']['mean']}")
    else:
        print(f"{task.dataset_name}: MAE {task.scores['mae']['mean']}")

    task.df = None

2021-08-02 17:41:03 INFO     Loading dataset 'matbench_dielectric'...
2021-08-02 17:41:07 INFO     Dataset 'matbench_dielectric loaded.
2021-08-02 17:41:07 INFO     Recorded fold matbench_dielectric-0 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_dielectric-1 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_dielectric-2 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_dielectric-3 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_dielectric-4 successfully.
matbench_dielectric: MAE 0.32100674639486443
2021-08-02 17:41:07 INFO     Loading dataset 'matbench_jdft2d'...
2021-08-02 17:41:07 INFO     Dataset 'matbench_jdft2d loaded.
2021-08-02 17:41:07 INFO     Recorded fold matbench_jdft2d-0 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_jdft2d-1 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_jdft2d-2 successfully.
2021-08-02 17:41:07 INFO     Recorded fold matbench_jdft2d-3 successful

In [7]:
mb.to_file("results.json.gz")

2021-08-02 17:09:10 INFO     Successfully wrote MatbenchBenchmark to file 'results.json.gz'.


In [8]:
for task in mb.tasks:

    if task.dataset_name != "matbench_glass":
        continue

In [16]:
mb = MatbenchBenchmark(
    autoload=False, 
    subset=[
        'matbench_glass', 
    ],
)

for task in mb.tasks:
    continue
task.load()

with open(results_locs[task.dataset_name], "rb") as f:
    results = pickle.load(f)
    
for fold_ind, fold in enumerate(task.folds):

    # Classification tasks require different loading as two values are reported per prediction
    predictions = results["predictions"][fold_ind].values[:, 0]
    task.record(fold, predictions)
    
if task.metadata.task_type == CLF_KEY:
    print(f"{task.dataset_name}: Accuracy score {task.scores['accuracy']['mean']}")
else:
    print(f"{task.dataset_name}: MAE {task.scores['mae']['mean']}")

task.df = None

2021-08-02 17:41:18 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_glass']
2021-08-02 17:41:18 INFO     Loading dataset 'matbench_glass'...
2021-08-02 17:41:18 INFO     Dataset 'matbench_glass loaded.
2021-08-02 17:41:18 INFO     Recorded fold matbench_glass-0 successfully.


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [12]:
task.get_test_data(0).values

array(['Al', 'Al10Co31B9', 'Al10Ni33B7', ..., 'ZrFe5Co4', 'ZrFe7B2',
       'ZrNb7Si2'], dtype=object)

In [ ]:
predictions = results["

In [ ]:
results

{'targets': [        gfa
  id0       0
  id6       0
  id17      0
  id21      1
  id22      0
  ...     ...
  id5641    0
  id5651    0
  id5654    1
  id5655    1
  id5666    1
  
  [1136 rows x 1 columns],
          gfa
  id7       0
  id9       1
  id15      1
  id16      1
  id19      1
  ...     ...
  id5658    1
  id5660    1
  id5664    1
  id5665    1
  id5671    1
  
  [1136 rows x 1 columns],
          gfa
  id1       1
  id3       1
  id14      1
  id20      1
  id23      0
  ...     ...
  id5663    1
  id5668    1
  id5670    1
  id5673    1
  id5679    1
  
  [1136 rows x 1 columns],
          gfa
  id4       1
  id8       1
  id10      1
  id11      1
  id12      1
  ...     ...
  id5657    1
  id5667    1
  id5675    0
  id5676    1
  id5678    1
  
  [1136 rows x 1 columns],
          gfa
  id2       1
  id5       0
  id26      0
  id34      0
  id37      0
  ...     ...
  id5656    1
  id5669    1
  id5672    1
  id5674    1
  id5677    1
  
  [1136 rows x 1 columns]]

In [ ]:
task.folds

[0, 1, 2, 3, 4]

In [ ]:
task.load()

2021-08-02 13:39:40 INFO     Loading dataset 'matbench_expt_is_metal'...
2021-08-02 13:39:40 INFO     Dataset 'matbench_expt_is_metal loaded.


In [ ]:
task.get_test_data()

mbid
mb-expt-is-metal-0002          Ag(W3Br7)2
mb-expt-is-metal-0003    Ag0.5Ge1Pb1.75S4
mb-expt-is-metal-0007             Ag2GeS3
mb-expt-is-metal-0008            Ag2GeSe3
mb-expt-is-metal-0009            Ag2GeTe3
                               ...       
mb-expt-is-metal-4895             ZrInPd2
mb-expt-is-metal-4897               ZrIr2
mb-expt-is-metal-4902                ZrO2
mb-expt-is-metal-4903               ZrPd2
mb-expt-is-metal-4913               ZrSe2
Name: composition, Length: 984, dtype: object

In [ ]:
from matminer.datasets import load_dataset

In [ ]:
df = load_dataset("matbench_expt_is_metal")

In [ ]:
from modnet.matbench.benchmark import matbench_kfold_splits

/home/mevans/.local/conda/envs/modnet_38/lib/python3.8/site-packages/mpmath/ctx_mp_python.py:892: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:
/home/mevans/.local/conda/envs/modnet_38/lib/python3.8/site-packages/mpmath/ctx_mp_python.py:986: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:


In [ ]:
matbench_kfold_splits(classification=True)
